# CRIM Intervals:  Modules

### What You Can Do with this Notebook

* Find contrapuntal modules in a corpus of pieces

Read working with a corpus:  

https://github.com/HCDigitalScholarship/intervals/blob/rich_dev_22/tutorial/01_Introduction.md#importing-multiple-pieces-at-once-corpusbase


Corpus of Remote Files:

* The pieces are provided as a **list**, within **square brackets** and **separated by commas**.  
* The bracketed list is then contained within the parentheses of `CorpusBase()`
* For example: 

```
corpus = CorpusBase(['https://crimproject.org/mei/CRIM_Mass_0006_1.mei', 'https://crimproject.org/mei/CRIM_Mass_0006_2.mei', 'https://crimproject.org/mei/CRIM_Mass_0006_3.mei'])
```
Corpus of Local Files:

```
import glob
corpus_list = []
for name in glob.glob('Music_Files/*'):
    corpus_list.append(name)
corpus = CorpusBase(corpus_list)
```



### A. Import Intervals and Other Code


In [1]:
from intervals import * 
from intervals import main_objs
from IPython.display import display
from ipywidgets import interact
from pandas.io.json import json_normalize
from pyvis.network import Network
import altair as alt
import glob
import intervals
import intervals.visualizations as viz
import matplotlib.pyplot as plt
import os
import pandas as pd
import re
import requests
import seaborn as sns

MYDIR = ("saved_csv")
CHECK_FOLDER = os.path.isdir(MYDIR)

# If folder doesn't exist, then create it.
if not CHECK_FOLDER:
    os.makedirs(MYDIR)
    print("created folder : ", MYDIR)
else:
    print(MYDIR, "folder already exists.")
    
MUSDIR = ("Music_Files")
CHECK_FOLDER = os.path.isdir(MUSDIR)

# If folder doesn't exist, then create it.
if not CHECK_FOLDER:
    os.makedirs(MUSDIR)
    print("created folder : ", MUSDIR)
else:
    print(MUSDIR, "folder already exists.")

saved_csv folder already exists.
Music_Files folder already exists.


In [4]:
# define remote corpus
corpus = CorpusBase(['https://crimproject.org/mei/CRIM_Model_0008.mei',
                     'https://crimproject.org/mei/CRIM_Mass_0005_2.mei'])

In [3]:
# or a set of local files
corpus_list = []
for name in glob.glob('Music_Files/Ser*.*'):
    corpus_list.append(name)
corpus = CorpusBase(corpus_list)

Empty corpus created. Please import at least one score.


### D.1  Corpus Ngram Inventory

*  Get the `ngrams` for all of them.  
*  In this case:  modules of length "3", with diatonic simple intervals
*  Then combine them into one frame



* NB: use `ImportedPiece`, not `piece`!
* NB:  for `func` do **NOT** include the closing parentheses!

```
func = ImportedPiece.ngrams
list_of_modules = corpus.batch(func=func, kwargs={'n': 3, 'interval_settings': ('d', False, True)}, metadata=True)
title_of_output = pd.concat(list_of_melodic_ngrams)
```

Note that the output lists voice pairs by **staff position**.  In a four-voice piece, the lowest part would be 4, then 3, etc.  `4_3` represents the two lowest voices of the piece, and so on.


In [5]:
# remember to omit "()" after calling `ngrams`
# here we are using simple diatonic
func = ImportedPiece.ngrams
list_of_modules = corpus.batch(func=func, kwargs={'n': 3, 'interval_settings': ('d', True, True), 'offsets': 'last'}, metadata=True)
func2 = ImportedPiece.detailIndex
list_of_details = corpus.batch(func=func2, kwargs={'offset': True, 'df': list_of_modules})

module_corpus = pd.concat(list_of_details).fillna('-').dropna(how="all")

c = module_corpus['Composer']
t = module_corpus["Title"]
module_corpus.pop("Composer")
module_corpus.pop("Title")
module_corpus.insert(0, "Composer", c)
module_corpus.insert(1, "Title", t)
module_corpus

Composer                    Title  \
Measure Beat Offset                                              
4       1.0  24.0    Josquin Des Prés                Ave Maria   
        3.0  28.0    Josquin Des Prés                Ave Maria   
6       1.0  40.0    Josquin Des Prés                Ave Maria   
        3.0  44.0    Josquin Des Prés                Ave Maria   
8       1.0  56.0    Josquin Des Prés                Ave Maria   
...                               ...                      ...   
161     4.0  1342.0  Antoine de Févin  Missa Ave Maria: Gloria   
162     1.0  1344.0  Antoine de Févin  Missa Ave Maria: Gloria   
        4.0  1350.0  Antoine de Févin  Missa Ave Maria: Gloria   
        4.5  1351.0  Antoine de Févin  Missa Ave Maria: Gloria   
163     1.0  1352.0  Antoine de Févin  Missa Ave Maria: Gloria   

                                  4_3                 4_2                 4_1  \
Measure Beat Offset                                                             
4       1.0  24.0                   -                   -                   -   
        3.0  28.0                   -                   -                   -   
6       1.0  40.0                   -                   -                   -   
        3.0  44.0                   -                   -                   -   
8       1.0  56.0    12_4, 10_Held, 8                   -                   -   
...                               ...                 ...                 ...   
161     4.0  1342.0                 -                   -  10_-2, 11_Held, 10   
162     1.0  1344.0     3_-2, 5_-5, 8      3_-2, 8_-5, 10  11_Held, 10_-5, 15   
        4.0  1350.0                 -    8_-5, 10_Held, 9                   -   
        4.5  1351.0                 -  10_Held, 9_Held, 8                   -   
163     1.0  1352.0                 -  9_Held, 8_Held, 12                   -   

                                   3_2              3_1               2_1  \
Measure Beat Offset                                                         
4       1.0  24.0                    -                -  12_4, 10_Held, 8   
        3.0  28.0                    -                -   10_Held, 8_1, 8   
6       1.0  40.0       5_4, 3_Held, 1                -                 -   
        3.0  44.0       3_Held, 1_1, 1                -                 -   
8       1.0  56.0                    -                -                 -   
...                                ...              ...               ...   
161     4.0  1342.0                  -   8_2, 7_Held, 6    8_5, 4_Held, 3   
162     1.0  1344.0       1_2, 4_-2, 3  7_Held, 6_-2, 8   4_Held, 3_-3, 6   
        4.0  1350.0    4_-2, 3_Held, 2                -     3_-3, 6_-2, 7   
        4.5  1351.0  3_Held, 2_Held, 1                -     6_-2, 7_-2, 8   
163     1.0  1352.0  2_Held, 1_Held, 5                -      7_-2, 8_5, 4   

                     Date  
Measure Beat Offset        
4       1.0  24.0    1502  
        3.0  28.0    1502  
6       1.0  40.0    1502  
        3.0  44.0    1502  
8       1.0  56.0    1502  
...                   ...  
161     4.0  1342.0  1515  
162     1.0  1344.0  1515  
        4.0  1350.0  1515  
        4.5  1351.0  1515  
163     1.0  1352.0  1515  

[1097 rows x 9 columns]

### D.2  Search Ngrams in the Corpus

* Note that the interval and pattern settings (diatonic/chromatic; compound/simple; length of ngrams) for this tool are set in the request for **module_corpus** above.

In [6]:
from IPython.display import display, HTML
display(HTML("<style>div.output_scroll { height: 44em; }</style>"))
@interact
def new_ngram_search(my_search="", df = fixed(module_corpus)):
    df2 = module_corpus.copy()
    c = df['Composer']
    t = df["Title"]
    df2.pop("Composer")
    df2.pop("Title")
    df2.insert(0, "Composer", c)
    df2.insert(1, "Title", t)
    filtered_ngrams = df2[df2.apply(lambda x: x.astype(str).str.contains(my_search).any(), axis=1)].copy()
    
    pd.set_option('max_columns', None)
    return filtered_ngrams.fillna("-").reset_index().applymap(str).style.applymap(lambda x: "background: #ccebc4" if re.search(my_search, x) else "")

interactive(children=(Text(value='', description='my_search'), Output()), _dom_classes=('widget-interact',))